# 순차구매 상품분석 실습


## 1.환경설정 ##

### 1.1 분석에 필요한 library 설치, 호출 및 google drive 연결

In [ ]:
# pyspark 설치 (Python 에서 스파크 연결을 위한 라이브러리 설치)
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=ef35e8bbe6deb4f1df6c09378f960016349e12785c9c80b01f88ec19fc835f3c
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
# 스파크 환경에서 데이터 핸들링을 위한 라이브러리 로딩
from pyspark.sql import functions as F, SparkSession, Column
from pyspark.ml.fpm import PrefixSpan

In [ ]:
# 구글드라이브에 있는 데이터셋 연결을 위한 구글드라이브 세팅
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2.분석 환경 생성 ##

### 2.1 스파크 세션 생성

In [ ]:
# 스파크 세션 생성
spark = SparkSession.builder \
    .appName("Seq_Analysis") \
    .getOrCreate()

### 2.2스파크환경으로 데이터 로딩 및 확인 ###

In [ ]:
# (참고) spark.read.csv는 파일을 디렉토리 단위로 읽어드림, 디렉토리만 선택할 경우 해당 디릭토리의 모든 데이터를 읽어드림
# 디렉토리에 한글명 또는 띄어쓰기가 들어가면 에러 발생함. 최대한 경로를 간단히 영문으로 작성 필요
sp_df = spark.read.csv("/content/drive/MyDrive/CRM_네트워킹데이/data/data.csv", header = True)
sp_df.show()  # 데이터 형식이 다름

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [ ]:
# 데이터 컬럼 확인
sp_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



## 3.데이터 정제 및 변환

### 3.1 NULL값 제거 ###

In [ ]:
# sp_df.where(sp_df['Description'].isNotNull()).where(sp_df['Quantity'] > 0).where(sp_df['CustomerID'].isNotNull()).show()
sp_df = sp_df.where(sp_df['Description'].isNotNull()).where(sp_df['Quantity'] > 0).where(sp_df['CustomerID'].isNotNull())

### 3.2 데이터 변환 ###

In [ ]:
# 유저별 한번에 구매한 상품을 리스트로 만들기

sp_df.createOrReplaceTempView("df")  # 데이터 프레임을 임시 view로 만듦 SQL에서 사용할 table

item_df =\
spark.sql(
    """select CustomerID, InvoiceNo, collect_list(Description) as items
    from df
    group by CustomerID, InvoiceNo
    order by InvoiceNo
    """
)
item_df.show()

+----------+---------+--------------------+
|CustomerID|InvoiceNo|               items|
+----------+---------+--------------------+
|     17850|   536365|[WHITE HANGING HE...|
|     17850|   536366|[HAND WARMER UNIO...|
|     13047|   536367|[ASSORTED COLOUR ...|
|     13047|   536368|[JAM MAKING SET W...|
|     13047|   536369|[BATH BUILDING BL...|
|     12583|   536370|[ALARM CLOCK BAKE...|
|     13748|   536371|[PAPER CHAIN KIT ...|
|     17850|   536372|[HAND WARMER RED ...|
|     17850|   536373|[WHITE HANGING HE...|
|     15100|   536374|[VICTORIAN SEWING...|
|     17850|   536375|[WHITE HANGING HE...|
|     15291|   536376|[HOT WATER BOTTLE...|
|     17850|   536377|[HAND WARMER RED ...|
|     14688|   536378|[JUMBO BAG PINK P...|
|     17809|   536380|[JAM MAKING SET P...|
|     15311|   536381|[RETROSPOT TEA SE...|
|     16098|   536382|[INFLATABLE POLIT...|
|     18074|   536384|[WOOD BLACK BOARD...|
|     17420|   536385|[SET 3 WICKER OVA...|
|     16029|   536386|[WHITE WIR

In [ ]:
# 유저별 구매 순서별 구매한 상품을 리스트로 만들기

item_df.createOrReplaceTempView("item_df")

item_df =\
spark.sql(
    """select CustomerID, collect_list(items) as items
    from item_df
    group by CustomerID
    """
)
item_df.show(truncate = 100)

+----------+----------------------------------------------------------------------------------------------------+
|CustomerID|                                                                                               items|
+----------+----------------------------------------------------------------------------------------------------+
|     12347|[[BLACK CANDELABRA T-LIGHT HOLDER, AIRLINE BAG VINTAGE JET SET BROWN, COLOUR GLASS. STAR T-LIGHT ...|
|     12349|[[PARISIENNE CURIO CABINET, SWEETHEART WALL TIDY , PINK  HEART SHAPE LOVE BUCKET , GINGHAM HEART ...|
|     12350|[[CHOCOLATE THIS WAY METAL SIGN, METAL SIGN NEIGHBOURHOOD WITCH , RETRO MOD TRAY, RETRO PLASTIC E...|
|     12354|[[JUMBO BAG ALPHABET, CHARLOTTE BAG APPLES DESIGN, CHARLOTTE BAG VINTAGE ALPHABET , CHILDRENS CUT...|
|     12355|[[GROW A FLYTRAP OR SUNFLOWER IN TIN, ICE CREAM SUNDAE LIP GLOSS, DOUGHNUT LIP GLOSS , NOVELTY BI...|
|     12356|[[BAKING SET 9 PIECE RETROSPOT , WHITE HEART CONFETTI IN TUBE, LAVENDER SCEN

## 4.모델링 ##

In [ ]:
# 순차 연관성 분석 모델 생성
model = PrefixSpan(minSupport=0.05,
                   maxPatternLength=5,
                   maxLocalProjDBSize=32000000,
                   sequenceCol= "items")

In [ ]:
# 모델링 결과 저장
freq_data = model.findFrequentSequentialPatterns(item_df)

In [ ]:
freq_data.show(truncate = 100)

+-------------------------------------+----+
|                             sequence|freq|
+-------------------------------------+----+
|              [[RED RETROSPOT WRAP ]]| 238|
|                [[PARTY METAL SIGN ]]| 236|
|     [[GIANT 50'S CHRISTMAS CRACKER]]| 260|
|        [[CHARLOTTE BAG SUKI DESIGN]]| 249|
| [[SET OF 12 MINI LOAF BAKING CASES]]| 250|
|[[SET OF 6 SPICE TINS PANTRY DESIGN]]| 393|
|         [[LUNCH BAG VINTAGE DOILY ]]| 319|
|       [[FELTCRAFT 6 FLOWER FRIENDS]]| 337|
|       [[FELTCRAFT BUTTERFLY HEARTS]]| 287|
|  [[PINK FAIRY CAKE CHILDRENS APRON]]| 250|
|         [[REGENCY TEA PLATE ROSES ]]| 224|
| [[RED HANGING HEART T-LIGHT HOLDER]]| 385|
|               [[RED KITCHEN SCALES]]| 292|
|          [[CREAM HEART CARD HOLDER]]| 274|
|         [[REGENCY CAKESTAND 3 TIER]]| 881|
|  [[NATURAL SLATE HEART CHALKBOARD ]]| 587|
|        [[JUMBO BAG DOILEY PATTERNS]]| 299|
|         [[HOME BUILDING BLOCK WORD]]| 360|
| [[PLASTERS IN TIN VINTAGE PAISLEY ]]| 315|
|         

In [ ]:
freq_data= freq_data.withColumn("length",F.size(F.col("sequence")))

In [ ]:
 freq_data_final = freq_data.where(F.col("length") >= 2).where(F.col("freq") >= 200).orderBy(F.col("freq"),ascending=False)
 freq_data_final.show(truncate = 100)

+----------------------------------------------------------------------------------------------------+----+------+
|                                                                                            sequence|freq|length|
+----------------------------------------------------------------------------------------------------+----+------+
|                        [[WHITE HANGING HEART T-LIGHT HOLDER], [WHITE HANGING HEART T-LIGHT HOLDER]]| 405|     2|
|                                            [[REGENCY CAKESTAND 3 TIER], [REGENCY CAKESTAND 3 TIER]]| 330|     2|
|                                              [[JUMBO BAG RED RETROSPOT], [JUMBO BAG RED RETROSPOT]]| 312|     2|
|                                  [[ASSORTED COLOUR BIRD ORNAMENT], [ASSORTED COLOUR BIRD ORNAMENT]]| 291|     2|
|                                                                  [[PARTY BUNTING], [PARTY BUNTING]]| 280|     2|
|                                              [[LUNCH BAG RED RETROSPOT], [LUNC

In [ ]:
freq_data_final.toPandas().to_csv("/content/drive/MyDrive/CRM_네트워킹데이/result.csv")

참조 :

*   kaggle : https://www.kaggle.com/code/hanadool/sequential-analysis-prefixspan-using-pyspark
*   reference : https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.fpm.PrefixSpan